In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, concatenate, Lambda, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop,SGD,Nadam, Adagrad, Adadelta, Ftrl
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.initializers import Constant ,Orthogonal, RandomNormal, VarianceScaling, Ones, Zeros
from tensorflow.keras.constraints import Constraint, UnitNorm
from keras.callbacks import Callback, TerminateOnNaN, ModelCheckpoint
from sksurv.metrics import concordance_index_censored as concordance
import math
import time
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

### Data import

In [ ]:
Xdf = pd.read_csv('processedExpData.csv',index_col=0)

In [ ]:
ydf = pd.read_csv('yData.csv',index_col=0)

In [ ]:
cancer = pd.read_csv('cancerData.csv',index_col=0)

### Model and Nested Crossval Functions

In [ ]:
#This is the TFCox class

class TFCox():
    def __init__(self, seed=42,norm=False,optimizer='Adam',l1_ratio=1,lbda=0.0001,
                 max_it=50,learn_rate=0.001,momentum=0.1,stop_if_nan=True,stop_at_value=False, cscore_metric=False,suppress_warnings=True,verbose=0):
        
        self.max_it = max_it
        self.tnan = stop_if_nan
        self.tcscore = stop_at_value
        self.lr=learn_rate
        self.cscore=cscore_metric
        np.random.seed(seed)
        tf.random.set_seed(seed)
        self.op = optimizer
        self.l1r = l1_ratio
        self.lbda=lbda
        self.norm = norm
        self.verbose=verbose
        if suppress_warnings == True:
            import warnings
            warnings.filterwarnings('ignore')
       
        if optimizer == 'Adam':
            self.opt = Adam(learn_rate)
        if optimizer == 'SGD':
            self.opt = SGD(learn_rate)
        if optimizer == 'SGDmomentum':
            self.opt = SGD(learn_rate,momentum=momentum)
        if optimizer == 'RMSprop':
            self.opt = RMSprop(learn_rate)
        if optimizer == 'Ftrl':
            self.opt = Ftrl(learn_rate,l1_regularization_strength=l1_ratio*lbda*100, l2_regularization_strength=(1-l1_ratio)*lbda*100)
            #self.opt = Ftrl(learn_rate,l1_regularization_strength=1, l2_regularization_strength=0)
            
    def coxloss(self, state):        
        def loss(y_true, y_pred):  
                return -K.mean((y_pred - K.log(tf.math.cumsum(K.exp(y_pred),reverse=True,axis=0)+0.0001))*state,axis=0)

        return loss

    def cscore_metric(self, state):
        def loss(y_true,y_pred):
            con = 0
            dis = 0
            for a in range(len(y_pred)):
                for b in range(a+1,len(y_pred)):                                       
                        if (y_pred[a]>y_pred[b])  & (y_pred[a]*state[a]!=0):
                            con+=1
                            
                        elif (y_pred[a]<y_pred[b])  & (y_pred[a]*state[a]!=0):
                            dis+=1
            return     con/(con+dis)
        return loss
 
    
    def fit(self, X,state,time):
        from tensorflow.python.framework.ops import disable_eager_execution
        disable_eager_execution()
        K.clear_session()
       
        
        
        self.time = np.array(time)  
        self.newindex = pd.DataFrame(self.time).sort_values(0).index
        self.X = (pd.DataFrame(np.array(X)).reindex(self.newindex))                      
        self.state = np.array(pd.DataFrame(np.array(state)).reindex(self.newindex))
        self.time  = np.array(pd.DataFrame(np.array(time)).reindex(self.newindex))                       
        inputsx = Input(shape=(self.X.shape[1],)) 
        state = Input(shape=(1,))
        
#         if self.op == 'Ftrl':
#             if self.norm==True:
#                 out = BatchNormalization()(inputsx)
#                 out = Dense(1,activation='linear', use_bias=False)(out)
#             else:
#                 out = Dense(1,activation='linear', use_bias=False)(inputsx)
                
#         else:
        if self.norm==True:
            out = BatchNormalization()(inputsx)
            if self.op!='Ftrl':
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(out)
            else:
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                   use_bias=False)(out)
        else:
            if self.op!='Ftrl':
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(inputsx)
            else:
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                   use_bias=False)(inputsx)
        model = Model(inputs=[inputsx, state], outputs=out)
        if (self.tcscore != False) or (self.cscore==True) :
            model.compile(optimizer=self.opt ,
                          loss=self.coxloss(state) , metrics=[self.cscore_metric(state)],
                          experimental_run_tf_function=False)
        else:
            model.compile(optimizer=self.opt ,
                          loss=self.coxloss(state) ,
                          experimental_run_tf_function=False)
        
        self.model=model
        if self.verbose==1:
            print(self.model.summary())

        self.loss_history_ = []
        for its in range(self.max_it):
            self.temp_weights = self.model.get_weights()
           
            tr = self.model.train_on_batch([self.X, self.state],np.zeros(self.state.shape))
           
            self.loss_history_.append(tr) 
            
            if self.verbose == 1:
                if (self.tcscore != False) or (self.cscore==True) :
                    print('loss:', self.loss_history_[-1][0],' C-score: ',self.loss_history_[-1][1] )
                else:
                    print('loss:', self.loss_history_[-1] )
            
            if self.tcscore != False:
                if self.loss_history_[-1][1]>=self.tcscore:
                    print('Terminated early because concordance >=' +str(self.tcscore)+ ' as set by stop_at_value flag.')
                    break
            if (self.tcscore != False) or (self.cscore==True) :
                if (math.isnan(self.loss_history_[-1][0]) or math.isinf(self.loss_history_[-1][0])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            else:
                if (math.isnan(self.loss_history_[-1]) or math.isinf(self.loss_history_[-1])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            
        self.beta_ = self.model.get_weights()[-1]

    def predict(self,X):
        preds = self.model.predict([X,np.zeros(len(X))])

        return preds

In [ ]:
#This is a function that runs stratified nested shuffle splits using the TFCox class

def nestedshuffle(lbda=[0.2], l1_r=1,learn_rate=[0.001],max_it=[50],optimizer=['Adam'] ,outloop=5,inloop=5,verbose=1):
    i=0
    np.random.seed(42)
    t0=time.time()
    index_map = pd.DataFrame(range(len(Xdf)),index=Xdf.index)

    outres_train = {}
    outres = {}
    inner_best = {}
    inner_best_train={}
    inner_var = {}
    errors = {}
    preds={}
    weights={}
    timeO = {}
    
       
    
    for out in range(outloop):
        
        t1=time.time()

        totres_train = pd.DataFrame()
        totres = pd.DataFrame()
        totres_var = pd.DataFrame()
        idx=[]
        for cnc in list(cancer):
            for st in [0,1]:
                np.random.seed(out)
                idx.extend(index_map.loc[ np.random.choice(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)].index, 
                                        size=int(0.8*len(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)])),replace=False)][0].tolist())


        idx = np.sort(np.ravel(np.array(idx)))

        SS = StandardScaler()
        X_train = Xdf.iloc[idx,:]
        X_train  = pd.DataFrame(SS.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
        X_test = Xdf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        X_test  = pd.DataFrame(SS.transform(X_test),index=X_test.index,columns=X_test.columns)
        y_train = ydf.iloc[idx,:]
        y_test = ydf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        can_train = cancer.iloc[idx,:]
        can_test = cancer.iloc[[x for x in range(len(Xdf)) if x not in idx],:]

        
        for lb in lbda:
            for op in optimizer:
                for lr in learn_rate:
                    for it in max_it:

                        t1=time.time()
                        results = pd.DataFrame()
                        trainresults = pd.DataFrame()
                        index_map2 = pd.DataFrame(range(len(X_train)),index=X_train.index)
                        for b in range(inloop):
                            t2=time.time()
                            K.clear_session()


                            idx2 = []
                            for cnc in list(cancer):
                                for st in [0,1]:
                                    np.random.seed(b)
                                    idx2.extend(index_map2.loc[ np.random.choice(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)].index, 
                                                            size=int(0.75*len(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)])),replace=False)][0].tolist())


                            idx2 = np.sort(np.ravel(np.array(idx2)))
                            SS = StandardScaler()
                            X_train2 = X_train.iloc[idx2,:]

                            X_train2  = pd.DataFrame(SS.fit_transform(X_train2),index=X_train2.index,columns=X_train2.columns)
                            X_test2 = X_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                            X_test2  = pd.DataFrame(SS.transform(X_test2),index=X_test2.index,columns=X_test2.columns)
                            y_train2 = y_train.iloc[idx2,:]
                            y_test2 = y_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                            can_train2 = can_train.iloc[idx2,:]
                            can_test2 = can_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]

                            resdict = {}
                            trainresdict = {}
                            for a in list(cancer):

                                Xt2 = X_train2[can_train2[a]==1]
                                Xts2 = X_test2[can_test2[a]==1]




                                K.clear_session()


                                model = TFCox(lbda = lb,optimizer=op,max_it=it,learn_rate=lr,l1_ratio=l1_r,verbose=verbose)


                                model.fit(Xt2,y_train2['vital_status'][can_train2[a]==1],y_train2['time'][can_train2[a]==1])


                                train_pred =  model.predict(Xt2).flatten()
                                pred = model.predict(Xts2).flatten()


                                resdict[a] = pred
                                trainresdict[a] = train_pred

                            res = pd.DataFrame()
                            trainres = pd.DataFrame()

                            for a in list(cancer):
                              #  print(trainresdict[a].shape)
                               # print(y_train2['vital_status'][can_train2[a]==1].astype('bool').shape)
                                try:

                                    trainres.loc[a,0]= concordance(y_train2['vital_status'][can_train2[a]==1].astype('bool'),y_train2['time'][can_train2[a]==1],trainresdict[a])[0]
                                except:

                                    trainres.loc[a,0] = 'fail'

                                try:
                                    res.loc[a,0]= concordance(y_test2['vital_status'][can_test2[a]==1].astype('bool'),y_test2['time'][can_test2[a]==1],resdict[a])[0]

                                except:
                                    res.loc[a,0] = 'fail'


                            print(i,'/',len(lbda) *outloop*(inloop+1) )
                            print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
                            results[str(b)] = res[0]
                            trainresults[str(b)] = trainres[0]
                            i+=1

                        totres_var[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = results.var(1)
                        totres[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = results.mean(1)
                        totres_train[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = trainresults.mean(1)


        inner_best[out] = totres
        inner_best_train[out] = totres_train
        inner_var[out] = totres_var
        outres[out] = pd.DataFrame()
        outres_train[out] = pd.DataFrame()
        timeO[out] = pd.DataFrame()
        
        
        
        for prms in list(inner_best[out]):
            weights[str(out)+str(prms)] = pd.DataFrame()
            K.clear_session()
            params = dict(prms)


            resdict = {}
            trainresdict = {}
            for a in list(cancer):
                
                Xt = X_train[can_train[a]==1]
                Xts = X_test[can_test[a]==1]
                
                
                t5=time.time()
             


                K.clear_session()

                model = TFCox(lbda = params['lambda'],optimizer=params['opt'],max_it=params['iter'],learn_rate=params['rate'],l1_ratio=l1_r,verbose=verbose)


                model.fit(Xt,y_train['vital_status'][can_train[a]==1],y_train['time'][can_train[a]==1])


                train_pred =  model.predict(Xt).flatten()
                pred = model.predict(Xts).flatten()

                weights[str(out)+str(prms)][a]=model.model.get_weights()
                resdict[a] = pred
                trainresdict[a] = train_pred

                #timeO[out].loc[a,frozenset(params.items())] = time.time()-t5
                
            res = pd.DataFrame()
            trainres = pd.DataFrame()
            for a in list(cancer):
                try:

                    trainres.loc[a,0]= concordance(y_train['vital_status'][can_train[a]==1].astype('bool'),y_train['time'][can_train[a]==1],trainresdict[a])[0]
                except:

                    trainres.loc[a,0] = 'fail'
                try:
                    res.loc[a,0]= concordance(y_test['vital_status'][can_test[a]==1].astype('bool'),y_test['time'][can_test[a]==1],resdict[a])[0]

                except:
                    res.loc[a,0] = 'fail'

            print(i,'/',len(lbda) *outloop*(inloop+1) )
            print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
            i+=1
            outres[out][frozenset(params.items())] = res[0]
            outres_train[out][frozenset(params.items())] = trainres[0]
          
    return outres , outres_train,inner_best,inner_best_train,inner_var,weights#,timeO

### Running the model

In [ ]:
# This is the geometric range of lambdas
vals = [float(np.format_float_positional(1 * 0.75**i, precision=2, unique=False, fractional=False, trim='k')) for i in range(33)]+[0]


In [ ]:
# Nested shuffle splits with LASSO Regularization and FTRL optimizer
lassoFtO1,lassoFtO2,lassoFtI1,lassoFtI2,lassoFtvar,lassoFtwt= nestedshuffle(lbda=vals,l1_r=1,optimizer=['Ftrl'],learn_rate=[0.001],max_it=[150] ,outloop=5, inloop=5,verbose=0)

In [ ]:
# Nested shuffle splits with elnet Regularization and FTRL optimizer
elnetFtO1,elnetFtO2,elnetFtI1,elnetFtI2,elnetFtvar,elnetFtwt= nestedshuffle(lbda=vals,l1_r=0.5,optimizer=['Ftrl'],learn_rate=[0.001],max_it=[150] ,outloop=5, inloop=5,verbose=0)

In [ ]:
# Nested shuffle splits with LASSO Regularization and Adam optimizer
lassoAdO1,lassoAdO2,lassoAdI1,lassoAdI2,lassoAdvar,lassoAdwt= nestedshuffle(lbda=vals,l1_r=1,optimizer=['Adam'],learn_rate=[0.001],max_it=[50] ,outloop=5, inloop=5,verbose=0)

In [ ]:
# Nested shuffle splits with elnet Regularization and Adam optimizer
elnetAdO1,elnetAdO2,elnetAdI1,elnetAdI2,elnetAdvar,elnetAdwt= nestedshuffle(lbda=vals,l1_r=0.5,optimizer=['Adam'],learn_rate=[0.001],max_it=[50] ,outloop=5, inloop=5,verbose=0)

# Finding the mean concordance from the nested crossval

In [ ]:
lassoresFt = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(lassoFtI1[b].index):
        
        
        lassoresFt.loc[a,b] =  lassoFtO1[b][lassoFtI1[b].idxmax(1)[a]][a]

In [ ]:
elnetresFt = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(elnetFtI1[b].index):
        
        
        elnetresFt.loc[a,b] =  elnetFtO1[b][elnetFtI1[b].idxmax(1)[a]][a]

In [ ]:
lassoresAd = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(lassoAdI1[b].index):
        
        
        lassoresAd.loc[a,b] =  lassoAdO1[b][lassoAdI1[b].idxmax(1)[a]][a]

In [ ]:
elnetresAd = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(elnetAdI1[b].index):
        
        
        elnetresAd.loc[a,b] =  elnetAdO1[b][elnetAdI1[b].idxmax(1)[a]][a]

In [ ]:
lassoresall = pd.DataFrame([lassoresFt.mean(1),lassoresAd.mean(1)]).transpose()
lassoresall.columns = ['FTRL','Adam']

In [ ]:
lassoresall['Cancer'] = list(lassoresall.index)

In [ ]:
#Display the lasso results
lassoresall

In [ ]:
elnetresall = pd.DataFrame([elnetresFt.mean(1),elnetresAd.mean(1)]).transpose()
elnetresall.columns = ['FTRL','Adam']

In [ ]:
elnetresall['Cancer'] = list(elnetresall.index)

In [ ]:
#Display the elastic net results
elnetresall